- 서포트 벡터 머신

분류 알고리즘

기본적으로 선형 분류지만 비선형에도 사용 가능

# SVC (당뇨병 데이터셋)

In [174]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [175]:
! wget https://raw.githubusercontent.com/devdio/datasets/refs/heads/main/diabetes.csv

--2024-12-30 08:20:29--  https://raw.githubusercontent.com/devdio/datasets/refs/heads/main/diabetes.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23873 (23K) [text/plain]
Saving to: ‘diabetes.csv.5’

diabetes.csv.5      100%[===================>]  23.31K  --.-KB/s    in 0.001s  

2024-12-30 08:20:29 (16.3 MB/s) - ‘diabetes.csv.5’ saved [23873/23873]



In [176]:
df = pd.read_csv("/content/diabetes.csv")
df.shape

(768, 9)

In [177]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [178]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Pregnancies,768.0,3.845052,3.369578,0.000,1.00000,3.0000,6.00000,17.00
Glucose,768.0,120.894531,31.972618,0.000,99.00000,117.0000,140.25000,199.00
BloodPressure,768.0,69.105469,19.355807,0.000,62.00000,72.0000,80.00000,122.00
SkinThickness,768.0,20.536458,15.952218,0.000,0.00000,23.0000,32.00000,99.00
Insulin,768.0,79.799479,115.244002,0.000,0.00000,30.5000,127.25000,846.00
BMI,768.0,31.992578,7.884160,0.000,27.30000,32.0000,36.60000,67.10
DiabetesPedigreeFunction,768.0,0.471876,0.331329,0.078,0.24375,0.3725,0.62625,2.42
Age,768.0,33.240885,11.760232,21.000,24.00000,29.0000,41.00000,81.00
Outcome,768.0,0.348958,0.476951,0.000,0.00000,0.0000,1.00000,1.00


In [179]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


### 범주형 변수

In [180]:
df['Outcome'].value_counts()

,count
Outcome,
0,500
1,268


### 숫자형

In [181]:
df['Pregnancies'].value_counts()

,count
Pregnancies,
1,135
0,111
2,103
3,75
4,68
5,57
6,50
7,45
8,38


## 3. 결측치

In [182]:
df.isna().sum()

,0
Pregnancies,0
Glucose,0
BloodPressure,0
SkinThickness,0
Insulin,0
BMI,0
DiabetesPedigreeFunction,0
Age,0
Outcome,0


##4. 이상치

In [183]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome'],
      dtype='object')

In [184]:
df_tmp = df[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin','BMI']]
df_tmp.head()

,Glucose,BloodPressure,SkinThickness,Insulin,BMI
0,148,72,35,0,33.6
1,85,66,29,0,26.6
2,183,64,0,0,23.3
3,89,66,23,94,28.1
4,137,40,35,168,43.1


In [185]:
# 0이 몇개씩 들어가있는지 확인
for i, col in enumerate(df_tmp.columns):
  print(col, len(df[df[col]==0]))

Glucose 5
BloodPressure 35
SkinThickness 227
Insulin 374
BMI 11


##5. 데이터 분리

In [186]:
X = df.drop('Outcome', axis = 1)
y = df['Outcome']

In [187]:
from sklearn.model_selection import train_test_split

# 데이터셋을 균형있게 나눌 때 stratify
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42, stratify=y)

In [188]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((614, 8), (154, 8), (614,), (154,))

In [189]:
y_train.value_counts()

,count
Outcome,
0,400
1,214


##6.이상치 처리

In [190]:
for i, col in enumerate(df_tmp.columns):
  print(col, len(df[df[col]==0]))

Glucose 5
BloodPressure 35
SkinThickness 227
Insulin 374
BMI 11


In [191]:
df_tmp.columns

Index(['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI'], dtype='object')

In [192]:
X_train[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin','BMI']] = X_train[['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin','BMI']].replace(0, np.nan)
X_train

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
353,1,90.0,62.0,12.0,43.0,27.2,0.580,24
711,5,126.0,78.0,27.0,22.0,29.6,0.439,40
373,2,105.0,58.0,40.0,94.0,34.9,0.225,25
46,1,146.0,56.0,NaN,NaN,29.7,0.564,29
682,0,95.0,64.0,39.0,105.0,44.6,0.366,22
...,...,...,...,...,...,...,...,...
451,2,134.0,70.0,NaN,NaN,28.9,0.542,23
113,4,76.0,62.0,NaN,NaN,34.0,0.391,25
556,1,97.0,70.0,40.0,NaN,38.1,0.218,30
667,10,111.0,70.0,27.0,NaN,27.5,0.141,40


In [193]:
X_train.isna().sum()

,0
Pregnancies,0
Glucose,4
BloodPressure,23
SkinThickness,175
Insulin,290
BMI,9
DiabetesPedigreeFunction,0
Age,0


In [194]:
cols = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin','BMI']

In [195]:
mean_dict = {}

for col in cols:
  u = X_train[col].mean()
  X_train[col] = X_train[col].fillna(u)
  mean_dict[col] = u

In [196]:
mean_dict

{'Glucose': 121.70163934426229,
 'BloodPressure': 72.14551607445009,
 'SkinThickness': 29.059225512528474,
 'Insulin': 149.07716049382717,
 'BMI': 32.44892561983471}

In [197]:
X_train.isna().sum()

,0
Pregnancies,0
Glucose,0
BloodPressure,0
SkinThickness,0
Insulin,0
BMI,0
DiabetesPedigreeFunction,0
Age,0


##7.스케일링

In [198]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
X_train_s = ss.fit_transform(X_train)
X_train_s

array([[-0.85135507, -1.05753334, -0.82718656, ..., -0.76979977,
         0.31079384, -0.79216928],
       [ 0.35657564,  0.14338879,  0.47732914, ..., -0.41781927,
        -0.11643851,  0.56103382],
       [-0.5493724 , -0.55714912, -1.15331549, ...,  0.35947099,
        -0.76486207, -0.70759409],
       ...,
       [-0.85135507, -0.8240207 , -0.17492871, ...,  0.82877832,
        -0.78607218, -0.28471812],
       [ 1.86648903, -0.35699543, -0.17492871, ..., -0.72580221,
        -1.01938346,  0.56103382],
       [ 0.05459296,  0.74384985, -1.15331549, ..., -0.43248513,
        -0.57700104,  0.30730824]])

In [199]:
# 인코딩 필요없음
y_train

,Outcome
353,0
711,0
373,0
46,0
682,0
...,...
451,1
113,0
556,0
667,1


In [200]:
type(X_train_s), type(y_train)
y_train = y_train.to_numpy()
type(X_train_s), type(y_train)

(numpy.ndarray, numpy.ndarray)

In [201]:
X_train_s

array([[-0.85135507, -1.05753334, -0.82718656, ..., -0.76979977,
         0.31079384, -0.79216928],
       [ 0.35657564,  0.14338879,  0.47732914, ..., -0.41781927,
        -0.11643851,  0.56103382],
       [-0.5493724 , -0.55714912, -1.15331549, ...,  0.35947099,
        -0.76486207, -0.70759409],
       ...,
       [-0.85135507, -0.8240207 , -0.17492871, ...,  0.82877832,
        -0.78607218, -0.28471812],
       [ 1.86648903, -0.35699543, -0.17492871, ..., -0.72580221,
        -1.01938346,  0.56103382],
       [ 0.05459296,  0.74384985, -1.15331549, ..., -0.43248513,
        -0.57700104,  0.30730824]])

##8.학습

베이스모델

In [202]:
from sklearn.svm import SVC

clf = SVC(random_state=42)
clf.fit(X_train_s, y_train)

SVC(random_state=42)

In [203]:
clf.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': 42,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

##9.테스트

In [204]:
X_test.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
44,7,159,64,0,0,27.4,0.294,40
672,10,68,106,23,49,35.5,0.285,47
700,2,122,76,27,200,35.9,0.483,26
630,7,114,64,0,0,27.4,0.732,34
81,2,74,0,0,0,0.0,0.102,22


In [205]:
X_train

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
353,1,90.0,62.0,12.000000,43.00000,27.2,0.580,24
711,5,126.0,78.0,27.000000,22.00000,29.6,0.439,40
373,2,105.0,58.0,40.000000,94.00000,34.9,0.225,25
46,1,146.0,56.0,29.059226,149.07716,29.7,0.564,29
682,0,95.0,64.0,39.000000,105.00000,44.6,0.366,22
...,...,...,...,...,...,...,...,...
451,2,134.0,70.0,29.059226,149.07716,28.9,0.542,23
113,4,76.0,62.0,29.059226,149.07716,34.0,0.391,25
556,1,97.0,70.0,40.000000,149.07716,38.1,0.218,30
667,10,111.0,70.0,27.000000,149.07716,27.5,0.141,40


In [206]:
# 이상치 처리

X_test[cols] = X_test[cols].replace(0, np.nan)

for col in cols:
  u = mean_dict[col]
  X_test[col] = X_test[col].fillna(u)

##스케일링

In [207]:
X_test_s = ss.transform(X_test)
X_test_s

array([[ 0.96054099,  1.24423407, -0.6641221 , ..., -0.74046806,
        -0.55579092,  0.56103382],
       [ 1.86648903, -1.79143019,  2.76023161, ...,  0.44746611,
        -0.58306107,  1.15306018],
       [-0.5493724 ,  0.009953  ,  0.31426467, ...,  0.50612953,
         0.01688223, -0.6230189 ],
       ...,
       [-0.5493724 , -1.32440492, -1.64250887, ..., -0.57914367,
         3.70138246, -0.70759409],
       [ 0.05459296,  2.07820777,  0.47732914, ...,  0.66745392,
        -0.64669142, -0.20014293],
       [-0.85135507, -1.69135335,  0.47732914, ...,  0.11015147,
        -0.16794879, -1.04589487]])

In [208]:
y_test = y_test.to_numpy

In [209]:
X_test_s.shape

(154, 8)

In [211]:
y_pred = clf.predict(X_test_s)
y_pred

array([1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0])

##9.모델평가

In [233]:
from sklearn.metrics import accuracy_score, recall_score
from sklearn.metrics import precision_score, f1_score
from sklearn.metrics import confusion_matrix

def print_metrics(y_true, y_pred):
  acc = accuracy_score(y_true, y_pred)
  re = recall_score(y_true, y_pred)
  pre = precision_score(y_true, y_pred)
  f1 = f1_score(y_true, y_pred)

  print("acc:", acc)
  print("recall:", np.around(re, 4))
  print("precision:", np.around(pre, 4))
  print("f1:", np.around(f1, 4))

  cfm = confusion_matrix(y_true, y_pred)
  s = sns.heatmap(cfm, annot=True, cmap='Blues', fmt='d', cbar=False)
  s.set(xlabel='Prediction', ylabel='Actual')
  plt.show()

##모델튜닝

In [215]:
from sklearn.model_selection import GridSearchCV

clf = SVC(random_state=43)
params = {
    'C':[0.1, 1, 10, 100],
    'gamma':[1,0.1,0.01,0.001],
    'degree':[2,3],
    'kernel':['rbf', 'linear', 'poly']
}

In [216]:
gs = GridSearchCV(estimator=clf,
              param_grid=params,
              cv=5,
              n_jobs=-1,
              verbose=3,
              refit=True
              )

In [217]:
gs.fit(X_train_s, y_train)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


GridSearchCV(cv=5, estimator=SVC(random_state=43), n_jobs=-1,
             param_grid={'C': [0.1, 1, 10, 100], 'degree': [2, 3],
                         'gamma': [1, 0.1, 0.01, 0.001],
                         'kernel': ['rbf', 'linear', 'poly']},
             verbose=3)

In [221]:
# 베스트 결과가 나오는 파라미터 조합을 알 수 있음
# 피클 라이브러리 사용해서 저장
gs.best_params_

{'C': 100, 'degree': 2, 'gamma': 0.001, 'kernel': 'rbf'}

##11.모델 저장

In [222]:
import joblib
joblib.dump(clf, 'svc_bin_diabtes_parmas.pkl')

['svc_bin_diabtes_parmas.pkl']

In [ ]:
joblib.dump(clf, 'svc_bin_diabtes_parmas.pkl')

In [223]:
clf = joblib.load('svc_bin_diabtes_parmas.pkl')

In [226]:
clf.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': 43,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

##12.knn 모델

In [234]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors = 5)
# 학습시킬 때 fit에 데이터 집어넣어서 사용
clf.fit(X_train_s, y_train)

KNeighborsClassifier()

In [235]:
clf.score(X_train_s, y_train)

0.8387622149837134

In [237]:
y_train

array([0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,

In [239]:
X_train_s

array([[-0.85135507, -1.05753334, -0.82718656, ..., -0.76979977,
         0.31079384, -0.79216928],
       [ 0.35657564,  0.14338879,  0.47732914, ..., -0.41781927,
        -0.11643851,  0.56103382],
       [-0.5493724 , -0.55714912, -1.15331549, ...,  0.35947099,
        -0.76486207, -0.70759409],
       ...,
       [-0.85135507, -0.8240207 , -0.17492871, ...,  0.82877832,
        -0.78607218, -0.28471812],
       [ 1.86648903, -0.35699543, -0.17492871, ..., -0.72580221,
        -1.01938346,  0.56103382],
       [ 0.05459296,  0.74384985, -1.15331549, ..., -0.43248513,
        -0.57700104,  0.30730824]])

In [238]:
print_metrics(X_train_s, y_train)

ValueError: Classification metrics can't handle a mix of continuous-multioutput and binary targets